 # Opis notatnika
 Analiza przygotowana w poprzednim kroku została odebrana pozytywnie. W związku z tym, zostaliśmy poproszeni
 o przygotowanie raportu na podstawie danych z roku 2020. Ma być on później  wdrożony przez zespół IT na zewnętrzny
 serwis. Wybór padł na `Dash`.

 Zadanie wykonamy w dwóch krokach:  
 1. Znając wymagania raportu, stworzymy na bazie danych odpowiednie komponenty, które zostaną następnie wykorzystane do wizualizacji.  
 1. Stworzymy raport używając `Dash`.

 Ten notatnik skupia się na odpowiednim przygotowaniu bazy danych pod cele raportowe.
 
 W tej części projektu końcowego wcielasz się w rolę BI Engineera, który ma za zadanie stworzyć raport zgodny z wytycznymi biznesowymi dostarczonymi przez klienta.

 # Wymagania biznesowe
 Po prezentacji wyników analizy na niezależnym spotkaniu, zostały określone następujące obszary raportowania odsetka opóźnień lotów:  
 1. Wyświetlanie TOP 10 (w sensie najwyższego odsetka opóźnień) lotnisk na podstawie liczby odlotów, wyświetlona ma być również informacja o liczbie przylotów - widok `top_airports_by_departure`.  
 1. Wyświetlenie TOP 10 (w sensie najwyższego odsetka opóźnień) tras lotów. Przy czym istotna dla nas jest kolejność, przykładowo trasa (Warszawa, Paryż) jest inna niż (Paryż, Warszawa). Dodatkowym wymaganiem jest, aby minimalna liczba lotów odbytych na trasie wynosiła co najmniej 10000 przelotów - widok `top_reliability_roads`.  
 1. Porównanie roku 2019 vs. 2020, aby śledzić wpływ COVID na realizację lotów. Interesują nas podejścia:  
     - miesiąc do miesiąca, przykładowo odsetek opoźnień styczeń 2019 vs. odsetek opoźnień styczeń 2020, odsetek opoźnień luty 2019 vs. odsetek opoźnień luty 2020 itd. - widok `year_to_year_comparision`,  
     - dzień do dnia, przykładowo odsetek opoźnień wtorek 2019 vs. odsetek opoźnień wtorek 2020 - widok `day_to_day_comparision`.  
 1. Dzienny, czyli jak danego dnia, globalnie wyglądał wskaźnik opóźnień lotów samolotu, tj. odsetek opóźnień 01-01-2019, odsetek opóźnień 02-01-2019 itd.

 # Podejście techniczne do problemu
 Naszym celem będzie odseparowanie warstwy przygotowania danych (logika raportu) od warstwy prezentacyjnej (wizualizacja).
 Chcemy zapewnić, aby odpowiednie procesy zajmowały się tylko swoimi zadaniami. 
 
 > W tym podejściu warstwa prezentacyjna (wykres/raport) nie implementuje logiki biznesowej w celu przetwarzania danych. Innymi słowy, nie chcemy aby przykładowo agregacja była wykonywana w momencie tworzenia wizualizacji.

 To podejście będzie spójne ze współczesnym sposobem projektowania aplikacji. Ma to też dodatkowy benefit - ze względu na wolumen danych na bazie, nie musimy ich najpierw pobierać - warstwa logiczna je odpowiednio zagreguje i przekaże zdecydowanie mniejszą liczbę wierszy, co przyśpieszy działanie całości.

 > Logika działania tego notebooka jest zbieżna z tą, którą robiliśmy już na przykładzie `Inicjowania bazy danych` - warto mieć go pod ręką.

 # Przygotowanie bazy danych
 Na bazie danych, gdzie umieszczone są już dane, wszystko zawarte jest na schemacie `public`. Ponieważ zgodnie z wymaganiami otrzymujemy nowy obszar wykorzystania danych, stworzymy sobie schemat dedykowany - `reporting`.  
 Dalej stworzymy widoki, które odpowiedzą na zadane wcześniej pytania.

 > Stworzenie dedykowanego schematu ma więcej korzyści niż nam się wydaje, w ten sposób możemy łatwo wprowadzić zabezpieczenie na dane, które chcemy udostępniać.

 ## Stworzenie dedykowanego schematu
 W pliku `reporting.sql` napisz kwerendę, która stworzy (o ile już nie istnieje) schemat `reporting`.
 Ten temat nie był omawiany w trakcie trwania kursu, jednak łatwo można uzupełnić wiedzę czytając np. [ten](https://www.postgresqltutorial.com/postgresql-administration/postgresql-create-schema/) samouczek.

 # Aktualizacja bazy danych
 W tym miejscu odpowiednio skonfiguruj połączenie do bazy danych.

 Tutaj zaimportuj potrzebne biblioteki

In [ ]:
import psycopg2
import pandas as pd
from sqlalchemy import create_engine

 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [ ]:
username = 'postgres'
password = '9292'
host = 'localhost'
database = 'airlines'
port = 5432

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [ ]:
conn = psycopg2.connect(
    dbname=database,
    user=username,
    password=password,
    host=host,
    port=port
)
cursor = conn.cursor()

In [ ]:
# Sprawdź, czy połączenie działa
cur = conn.cursor()
cur.execute('SELECT version();')
print(cur.fetchone()[0])  # Wyświetl wersję PostgreSQL

 ## Wczytanie pliku `reporting.sql`
 Z katalogu `sql` wczytaj plik `reporting.sql`

In [ ]:
with open("C:/Users/User/OneDrive - AEI/Pulpit/Pliki_do_Portfolio_Lab/sql/reporting.sql", 'r', encoding='utf-8') as reporting_file:
    reporting_sql = reporting_file.read()

 W tym miejscu odpowiednio rozdziel zawartość pliku `reporting.sql` na mniejsze kwerendy używając `;`

In [ ]:
# Podziel zawartość na mniejsze kwerendy, używając średnika jako separatora
queries = reporting_sql.split(';')

# Wyświetl każdą kwerendę
for i, query in enumerate(queries):
    print(f"Kwerenda {i + 1}:\n{query.strip()}\n")

 W tym miejscu wykonaj każdą z kwerend, aby zainicjować strukturę bazy danych

In [ ]:
# Utworzenie kursora
cur = conn.cursor()
# Wykonanie kwerendy SQL
cur.execute("CREATE SCHEMA IF NOT EXISTS reporting")
# Potwierdzenie operacji i zakończenie połączenia
conn.commit()

# Definicja widoku reporting.flight
reporting_flight = """
CREATE OR REPLACE VIEW reporting.flight AS
SELECT
    *,
    CASE
        WHEN dep_delay_new > 0 THEN 1
        ELSE 0
    END AS is_delayed
FROM
    public.flight
WHERE
    cancelled = 0
"""

# Definicja widoku reporting.top_reliability_roads
reporting_top_reliability_roads = """
CREATE OR REPLACE VIEW reporting.top_reliability_roads AS
SELECT
    subq.origin_airport_id,
    a.name AS origin_airport_name,
    subq.dest_airport_id,
    b.name AS dest_airport_name,
    subq.year,
    subq.cnt,
    subq.reliability,
    DENSE_RANK() OVER (PARTITION BY subq.year ORDER BY subq.reliability DESC) AS nb
FROM
    (SELECT
        f.origin_airport_id,
        f.dest_airport_id,
        f.year,
        COUNT(*) AS cnt,
        100.0 * SUM(CASE WHEN f.dep_delay_new > 0 THEN 1 ELSE 0 END) / COUNT(*) AS reliability
    FROM
        public.flight AS f
    GROUP BY
        f.origin_airport_id,
        f.dest_airport_id,
        f.year
    HAVING
        COUNT(*) > 10000) AS subq
JOIN
    airport_list AS a ON subq.origin_airport_id = a.origin_airport_id
JOIN
    airport_list AS b ON subq.dest_airport_id = b.origin_airport_id
"""

# Definicja widoku reporting.year_to_year_comparision
reporting_year_to_year_comparision = """
CREATE OR REPLACE VIEW reporting.year_to_year_comparision AS
SELECT
    f.year,
    f.month,
    COUNT(*) AS flights_amount,
    100.0 * SUM(CASE WHEN f.dep_delay_new > 0 THEN 1 ELSE 0 END) / COUNT(*) AS reliability
FROM
    public.flight AS f
GROUP BY
    f.year,
    f.month
"""
# Definicja widoku reporting.day_to_day_comparision
reporting_day_to_day_comparision = """
CREATE OR REPLACE VIEW reporting.day_to_day_comparision AS
SELECT
    f.year,
    f.day_of_week,
    COUNT(*) AS flights_amount
FROM
    public.flight AS f
GROUP BY
    f.year,
    f.day_of_week
"""

# Definicja widoku reporting.day_by_day_reliability
reporting_day_by_day_reliability = """
CREATE OR REPLACE VIEW reporting.day_by_day_reliability AS
SELECT
    TO_DATE(CONCAT(f.year, '-', LPAD(f.month::text, 2, '0'), '-', LPAD(f.day_of_week::text, 2, '0')), 'YYYY-MM-DD') AS date,
    100.0 * SUM(CASE WHEN f.dep_delay_new > 0 THEN 1 ELSE 0 END) / COUNT(*) AS reliability
FROM
    public.flight AS f
GROUP BY
    f.year,
    f.month,
    f.day_of_month;
"""

 Zatwierdzenie wszystkich operacji na bazie, czyli stworzenie widoków

In [ ]:
cur = conn.cursor()

# Wykonanie zapytań SQL
queries = [
    reporting_flight,
    reporting_top_reliability_roads,
    reporting_year_to_year_comparision,
    reporting_day_to_day_comparision,
    reporting_day_by_day_reliability
]

for query in queries:
    cur.execute(query)

# Zatwierdzenie zmian i zamknięcie połączenia
conn.commit()

 ### Sprawdzenie

In [ ]:
# Ten kod chyba wygląda znajomo....
# istnienie widoków możemy sprawdzić tak samo jak tabele
def check_if_table_exists(table_name):
    msg = f"Sprawdzam czy istnieje tabela {table_name}"
    print(msg)

    query = f"select 1 from {table_name}"
    # jeżeli tabela nie istnieje, ten krok zwróci wyjątek
    cursor.execute(query)
    print('OK!')

In [ ]:
views_to_test = [
    'reporting.flight',
    'reporting.top_reliability_roads',
    'reporting.year_to_year_comparision',
    'reporting.day_to_day_comparision',
    'reporting.day_by_day_reliability'
]

In [ ]:
for view in views_to_test:
    check_if_table_exists(view)

In [ ]:
conn.close()
msg = "Wszystko wygląda OK :) Możesz przejść do kolejnego zadania."
print(msg)

 # Podsumownie
 W tym notatniku stworzyliśmy nowy schemat - `reporting`, którego zadaniem jest przygotowanie naszych danych
 do wizualizacji. Dalsza część pracy będzie polegała na wyświetleniu w wizualnie atrakcyjny sposób danych w interaktywnym raporcie stworzonym
 z pomocą `Dash`.